# DATA 335 - Winter 2025 - Lab 5

2025.02.25, 14:00-15:50, MS 521

In [1]:
import numpy as np
import pandas as pd

### Subset selection, transformers, pipelines, and grid search

In this exercise, we use the `data/auto_preprocessed.csv` dataset. Let `X` be the matrix of numerical (non-binary) features and let the target vector `y` be the `mpg` column.



In [61]:
auto = pd.read_csv("../data/auto_preprocessed.csv")
numerical_features = np.array(
    [
        "acceleration",
        "cylinders",
        "displacement",
        "horsepower",
        "weight",
        "year",
    ]
)
X = auto[numerical_features].copy()
y = auto["mpg"].copy()

#### Goal

For `p` in `[1, 2, 3, 4, 5]`, determine the subset of `numerical` of size `p` for which the linear model using precisely those features has the best predictive performance, as approximated by repeated 5-fold cross-validation.

We'll start by doing things "by hand" and then refactor to incorporate some useful scikit-learn idioms: **transformers**, **pipelines**, and **grid search**.

##### Task 1

Complete the implementation of the function `repeated_kfold_error` (fill in the `...`) whose goal is estimating the predictive error of a model using repeated $k$-fold cross validation.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error


def repeated_kfold_error(model, X, y, *, n_splits=5, n_repeats=20, random_state=None):
    n_fits = n_splits * n_repeats
    errors = np.zeros(n_fits)
    repeated_kfold = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats)
    for i, (train, test) in enumerate(repeated_kfold.split(X)):
        model.fit(..., ...)  # Your work here!
        errors[i] = mean_squared_error(..., ...)  # Your work here!
    return errors.mean()


# check
assert np.isclose(
    repeated_kfold_error(LinearRegression(), X, y, random_state=42).mean(),
    12.167089387362708,
)

Scikit-learn provides the convenience method `sklearn.model_selection.cross_val_score` for iterating over train/test splits arising from cross-validation, fitting models, and collecting prediction metrics.

Scikit-learn conforms to standard English usage by considering higher scores to be better. For measures of error, on the other hand, lower is better. Thus, the negative of a score is an error measure, and vice versa. In particular, the scores returned by calls to `cross_val_score` with the `scoring` keyword argument set to `neg_mean_squared_error` are the negatives of the test set MSEs.

##### Task 2

Fill in the `...` on the last three lines of the cell below to verify that the errors returned by `repeated_kfold_error` are the negatives of the scores returned by `repeated_kfold_score` when these functions are passed the same `random_state`.

In [ ]:
from sklearn.model_selection import cross_val_score


def repeated_kfold_score(model, X, y, *, n_splits=5, n_repeats=10, random_state=None):
    cv = RepeatedKFold(
        n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
    )
    scores = cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=cv)
    return scores


errors = repeated_kfold_error(LinearRegression(), X, y, random_state=...)
scores = repeated_kfold_score(LinearRegression(), X, y, random_state=...)
assert ...

##### Task 3

Complete the implementation of the function `find_best_features` (fill in the `...`) whose goal is to find the set of features among those in `feature_lists` minimizing predictive error, as estimated by repeated $k$-fold cross-validation.

The argument `feature_lists` is a list of lists of features of `X`. For example, `features_lists` might be the list of all pairs of features:

```python
feature_lists = [
    [f, g]
    for f in numerical_features
    for g in numerical_features
    if f < g
]
feature_lits
```

**Output:**
```text
[['acceleration', 'cylinders'],
 ['acceleration', 'displacement'],
 ['acceleration', 'horsepower'],
 ['acceleration', 'weight'],
 ['acceleration', 'year'],
 ['cylinders', 'displacement'],
 ['cylinders', 'horsepower'],
 ['cylinders', 'weight'],
 ['cylinders', 'year'],
 ['displacement', 'horsepower'],
 ['displacement', 'weight'],
 ['displacement', 'year'],
 ['horsepower', 'weight'],
 ['horsepower', 'year'],
 ['weight', 'year']]
```


In [ ]:
def find_best_features(
    model, X, y, feature_lists, n_splits=5, n_repeats=10, random_state=None
):
    best_features = []
    best_score = -np.inf
    for features in feature_lists:
        score = repeated_kfold_score(
            model,
            ...,  # Your work here!
            y,
            n_splits=n_splits,
            n_repeats=n_repeats,
            random_state=random_state,
        )
        if score > best_score:
            best_score = score
            best_features = features
    return best_features


# check
feature_lists = [
    [f, g] for f in numerical_features for g in numerical_features if f < g
]
best_features, best_score = find_best_features(
    LinearRegression(), X, y, feature_lists, random_state=42
)
assert best_features == ["weight", "year"]
assert np.isclose(best_score, -11.87058663029629)

##### Task 4

Write a function `enumerate_sublists(n)` a positive integer `n` as input and produces, as output, a list `S` of length `n + 1` such that `S[i]` is a list consisting of all sublists of `list(range(n))` of size `i`. In particular, `S[0]` should be `[[]]` and `S[n]` should be `[list(range(n))]`. Arrange for `S[i]` to be ordered lexicographically.

Flesh out the implementation below. If you don't like my approach (maybe you prefer recursion?), scap it and do your own thing.

In [ ]:
def enumerate_sublists(n):
    S = [[[]], [[i] for i in range(n)], *[[] for _ in range(2, n + 1)]]
    for i in range(2, n + 1):
        for s in S[i - 1]:
            S[i].extend(...)  # Your stuff here!
    return S


# check
assert enumerate_sublists(3) == [
    [[]],
    [[0], [1], [2]],
    [[0, 1], [0, 2], [1, 2]],
    [[0, 1, 2]],
]

##### Task 5

Embed `find_best_features` in a for-loop to find the optimal sets of numerical features of sizes 1 through 6.

In [ ]:
p = len(numerical_features)
sublists = enumerate_sublists(p)
for S in sublists[1:]:
    feature_lists = [numerical_features[s] for s in S]
    best_features, best_score = find_best_features(...)  # Your work here!
    print(best_features, best_score)

#### Transfomers and pipelines

**Transformers** are scikit-learn classes for transforming data, often part of a preprocessing step. Here's a simple transformer that extracts a given collection of columns from a dataset:

In [269]:
from sklearn.base import BaseEstimator, TransformerMixin


class FilterFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, features=None):
        self.features = features

    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            return self
        else:
            raise TypeError("X must be a pandas dataframe.")

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            return X.loc[:, self.features]
        else:
            raise TypeError("X must be a pandas dataframe.")


# Demonstration

F = FilterFeatures(features=["weight", "year"])
XX = F.fit_transform(X)
display(XX.head())
assert all(XX.columns == ["weight", "year"])

,weight,year
0,3504,70
1,3693,70
2,3436,70
3,3433,70
4,3449,70


Transformers and estimators (models) can be composed into **pipelines**. Pipelines can be fit and used for prediction just like any other scikit-learn model.

In [272]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(FilterFeatures(features=["weight", "year"]), LinearRegression())
model.fit(X, y)
mse = mean_squared_error(y, model.predict(X))
print(f"mse = {mse:.2f}")

mse = 11.70


#### Grid search

Grid search fits a collection of models parametrized by a **parameter grid** to the same dataset in order to compare their performance. The `sklearn.model_selection.GridSearchCV` class does this comparison using cross-validation. Our `FilterFeatures`-`LinearRegression` pipeline is parametrized by the `features` parameter of the `FilterFeatures` step. Below, I use `GridSearchCV` to identify an optimal feature set from a collection, just like we accomplished above manually:

In [258]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


model = make_pipeline(FilterFeatures(), LinearRegression())

feature_lists = [
    [f, g] for f in numerical_features for g in numerical_features if f < g
]
param_grid = {"filterfeatures__features": feature_lists}

search = GridSearchCV(
    model,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",
    cv=RepeatedKFold(random_state=42),
)

search.fit(X, y)

print(search.best_params_, search.best_score_)

{'filterfeatures__features': ['weight', 'year']} -11.87058663029629


##### Task 6

Refactor the function `find_best_features` from Task 3 to use `GridSearchCV` instead of hand-crafted the for loop. Call the new function `search_best_features`. Show that `find_best_features` and `search_best_features` give the same results.

In [ ]:
def search_best_features(
    model, X, y, feature_lists, n_splits=5, n_repeats=10, random_state=None
):
    pipeline = make_pipeline(FilterFeatures(), model)
    param_grid = ...  # Your work here!
    scoring = ...  # Your work here!
    cv = RepeatedKFold(
        n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
    )
    search = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring, cv=cv)
    search.fit(X, y)
    best_features = ...  # Your work here!
    best_score = ...  # Your work here!
    return best_features, best_score


# check
feature_lists = [
    [f, g] for f in numerical_features for g in numerical_features if f < g
]
a, b = find_best_features(LinearRegression(), X, y, feature_lists, random_state=42)
aa, bb = search_best_features(LinearRegression(), X, y, feature_lists, random_state=42)
assert a == aa
assert np.isclose(b, bb)

##### Task 7

Redo Task 5 using `search_best_features` from Task 6 in place of `find_best_features`.